Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((dataset.shape[0],-1)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [52]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Vars
  intermediateW = tf.Variable(tf.truncated_normal([image_size*image_size , hidden_nodes]))
  intermediateB = tf.Variable(tf.zeros([hidden_nodes]))
  finalW = tf.Variable(tf.truncated_normal([hidden_nodes , num_labels]))
  finalB = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0)
  #Comps
  intermediateO = tf.nn.relu(tf.matmul(tf_train_dataset , intermediateW) + intermediateB)
  regularizedI = tf.nn.l2_loss(intermediateW)
  logits = tf.matmul(intermediateO, finalW) + finalB
  lossF = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizedW = tf.nn.l2_loss(finalW)
  #lossF = tf.reduce_mean(lossT + (0.0001*regularizedW) +(0.0001*regularizedI) )
  
  learning_rate = tf.train.exponential_decay(0.1, global_step,50,0.5,staircase = True)
  #Optimizer
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(lossF, global_step=global_step)

  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(lossF)
    
    
  train_prediction = tf.nn.softmax(logits)

  valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset,intermediateW) + intermediateB)
  valid_prediction = tf.nn.softmax(tf.matmul(valid1, finalW) + finalB)

  test1 = tf.nn.relu(tf.matmul(tf_test_dataset,intermediateW) + intermediateB)
  test_prediction = tf.nn.softmax(tf.matmul(test1, finalW) + finalB)

In [53]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, lossF, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 382.992432
Minibatch accuracy: 10.9%
Validation accuracy: 22.9%
Minibatch loss at step 500: 22.224583
Minibatch accuracy: 84.4%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 35.200138
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 20.005577
Minibatch accuracy: 82.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 24.022125
Minibatch accuracy: 82.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 29.693258
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 38.330017
Minibatch accuracy: 78.1%
Validation accuracy: 78.2%
Test accuracy: 85.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Initialized
Minibatch loss at step 0: 296.210602
Minibatch accuracy: 18.0%
Validation accuracy: 28.0%
Minibatch loss at step 500: 29.900585
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 1000: 28.442293
Minibatch accuracy: 100.0%
Validation accuracy: 62.9%
Minibatch loss at step 1500: 27.055130
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Minibatch loss at step 2000: 25.735596
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Minibatch loss at step 2500: 24.480410
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Minibatch loss at step 3000: 23.286451
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Test accuracy: 68.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [125]:
batch_size = 128
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Vars
  intermediateW = tf.Variable(tf.truncated_normal([image_size*image_size , hidden_nodes]))
  intermediateB = tf.Variable(tf.zeros([hidden_nodes]))
  finalW = tf.Variable(tf.truncated_normal([hidden_nodes , num_labels]))
  global_step = tf.Variable(0)
  finalB = tf.Variable(tf.zeros([num_labels]))
  #Comps
  intermediateO = tf.nn.relu(tf.matmul(tf_train_dataset , intermediateW) + intermediateB)
  intermediateO = tf.nn.dropout(intermediateO,1)
  regularizedI = tf.nn.l2_loss(intermediateW)
  logits = tf.matmul(intermediateO, finalW) + finalB
  lossT = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizedW = tf.nn.l2_loss(finalW)
  lossF = tf.reduce_mean(lossT + (0.005*regularizedW) +(0.005*regularizedI) )
    
    
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(lossF)
    
    
  train_prediction = tf.nn.softmax(logits)

  valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset,intermediateW) + intermediateB)
  valid_prediction = tf.nn.softmax(tf.matmul(valid1, finalW) + finalB)

  test1 = tf.nn.relu(tf.matmul(tf_test_dataset,intermediateW) + intermediateB)
  test_prediction = tf.nn.softmax(tf.matmul(test1, finalW) + finalB)

In [126]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset+batch_size), :]
    batch_labels = train_labels[offset:(offset+batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, lossF, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2020.965088
Minibatch accuracy: 6.2%
Validation accuracy: 31.8%
Minibatch loss at step 500: 127.088669
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 10.973012
Minibatch accuracy: 82.0%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 1.365152
Minibatch accuracy: 90.6%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 0.616835
Minibatch accuracy: 90.6%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 0.622891
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.681681
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Test accuracy: 91.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
batch_size = 128
H1 = 1024
H2 = 50
beta = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #Vars
  w1 = tf.Variable(tf.truncated_normal([image_size*image_size , H1], stddev=np.sqrt(2.0 / (image_size * image_size))))
  w2 = tf.Variable(tf.truncated_normal([H1 , H2]))
  w3 = tf.Variable(tf.truncated_normal([H2 , num_labels]))
  b1 = tf.Variable(tf.zeros([H1]))
  b2 = tf.Variable(tf.zeros([H2]))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
  
  #Computations
  L1 = tf.nn.relu(tf.matmul(tf_train_dataset,w1) + b1)
  #L1 = tf.nn.dropout(L1,1)
  #R1 = tf.nn.l2_loss(W1)
  L2 = tf.nn.relu(tf.matmul(L1 , w2) + b2)
  #L2 = tf.nn.dropout(L2,1)
  #R2 = tf.nn.l2_loss(W2)
  logits = tf.matmul(L2 , w3) + b3
  lossF =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  #R3 = tf.nn.l2_loss(W3)
  #lossF = tf.reduce_mean(lossT + ((beta * R1) + (beta * R2) + (beta * R3)))
  #learning_rate = tf.train.exponential_decay(0.5, global_step,1000,0.96,staircase = True)
  #Optimizer
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(lossF, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(lossF) 

  train_prediction = tf.nn.softmax(logits)

  V1 = tf.nn.relu(tf.matmul(tf_valid_dataset,w1) + b1)
  V2 = tf.nn.relu(tf.matmul(V1,w2) + b2)
  valid_prediction = tf.nn.softmax(tf.matmul(V2, w3) + b3)

  T1 = tf.nn.relu(tf.matmul(tf_test_dataset,w1) + b1)
  T2 = tf.nn.relu(tf.matmul(T1,w2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(T2,w3) + b3)
  

SyntaxError: invalid syntax (<ipython-input-5-a269dfe825ff>, line 17)

In [134]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, lossF, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1180.542114
Minibatch accuracy: 6.2%
Validation accuracy: 10.4%
Minibatch loss at step 500: 2.312530
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 2.302702
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%


KeyboardInterrupt: 

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0)

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.8).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [122]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.917721
Minibatch accuracy: 10.9%
Validation accuracy: 14.9%
Minibatch loss at step 500: 1.190160
Minibatch accuracy: 75.8%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 1.079245
Minibatch accuracy: 77.3%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 0.673892
Minibatch accuracy: 84.4%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.763167
Minibatch accuracy: 84.4%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.930240
Minibatch accuracy: 82.8%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 0.938450
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Test accuracy: 86.9%


In [25]:
batch_size = 128
H1 = 1024
H2 = 50
beta = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #Vars
  W1 = tf.Variable(tf.truncated_normal([image_size*image_size , H1], stddev=np.sqrt(2.0 / (image_size * image_size))))
  W2 = tf.Variable(tf.truncated_normal([H1, H2], stddev=np.sqrt(2.0 / H1)))
  W3 = tf.Variable(tf.truncated_normal([H2 , num_labels], stddev=np.sqrt(2.0 / H2)))
  B1 = tf.Variable(tf.zeros([H1]))
  B2 = tf.Variable(tf.zeros([H2]))
  B3 = tf.Variable(tf.zeros([num_labels]))
  
  #Computations
  L1 = tf.nn.relu(tf.matmul(tf_train_dataset,W1) + B1)
  L1 = tf.nn.dropout(L1,1/2)
  #R1 = tf.nn.l2_loss(W1)
  L2 = tf.nn.relu(tf.matmul(L1 , W2) + B2)
  L2 = tf.nn.dropout(L2,1/2)
  #R2 = tf.nn.l2_loss(W2)
  logits = tf.matmul(L2 , W3) + B3
  lossF =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  #R3 = tf.nn.l2_loss(W3)
  #lossF = tf.reduce_mean(lossT + ((beta * R1) + (beta * R2) + (beta * R3)))
  #learning_rate = tf.train.exponential_decay(0.5, global_step,1000,0.96,staircase = True)
  #Optimizer
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(lossF, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(lossF) 

  train_prediction = tf.nn.softmax(logits)

  V1 = tf.nn.relu(tf.matmul(tf_valid_dataset,W1) + B1)
  V2 = tf.nn.relu(tf.matmul(V1,W2) + B2)
  valid_prediction = tf.nn.softmax(tf.matmul(V2, W3) + B3)

  T1 = tf.nn.relu(tf.matmul(tf_test_dataset,W1) + B1)
  T2 = tf.nn.relu(tf.matmul(T1,W2) + B2)
  test_prediction = tf.nn.softmax(tf.matmul(T2,W3) + B3)

In [26]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, lossF, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.563191
Minibatch accuracy: 14.1%
Validation accuracy: 26.1%
Minibatch loss at step 500: 0.528636
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.536215
Minibatch accuracy: 78.9%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.474741
Minibatch accuracy: 84.4%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 0.409127
Minibatch accuracy: 90.6%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 0.492175
Minibatch accuracy: 84.4%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 0.604675
Minibatch accuracy: 82.0%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 0.559787
Minibatch accuracy: 79.7%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.425020
Minibatch accuracy: 89.8%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 0.495892
Minibatch accuracy: 85.9%
Validation accuracy: 85.9%
Minibatch loss at step 5000: 0.509139
Minibatch accuracy: 84.4%
Validation accurac

KeyboardInterrupt: 